In [51]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver

In [56]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [2]:
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
    

Requestiong:  poloclub cnn-explainer
https://api.github.com/repos/poloclub/cnn-explainer/contributors
user "xiaohk" has 189 contributions
https://github-user.now.sh/?username=xiaohk


TypeError: 'Response' object is not subscriptable

### Saving author contributions to txt file

In [ ]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [30]:

repos = []
cursor.execute("SELECT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [32]:
len(repos)

20376

In [69]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [72]:
driver = webdriver.Firefox()

for repo in repos[504:1500]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        if name=="":
            name = "NULL"
        else:
            name = name.replace(" ", "_")
        with open(filename, 'a') as file:
            file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/kryvosheyev/xray-anomaly-detection
https://github.com/kryvosheyev
name: , bio: 
User kryvosheyev: commits 4 , changed lines + 3,897 - 21

https://github.com/wdlctc/darknet
https://github.com/AlexeyAB
name: Alexey, bio: 
User AlexeyAB: commits 974 , changed lines + 269,447 - 29,790
https://github.com/pjreddie
name: Joseph Redmon, bio: 
User pjreddie: commits 314 , changed lines + 119,047 - 37,962
https://github.com/cenit
name: Stefano Sinigardi, bio: 
User cenit: commits 112 , changed lines + 16,573 - 15,485
https://github.com/cyyever
name: cyy, bio: (list 1 2 3 4)
User cyyever: commits 21 , changed lines + 818 - 689
https://github.com/tinohager
name: Tino Hager, bio: A professional c# software developer
User tinohager: commits 14 , changed lines + 1,258 - 1,091
https://github.com/vinjn
name: Vinjn Zhang, bio: Senior GPU Architect @NVIDIA passionate about Visual Computing
User vinjn: commits 12 , changed lines + 122 - 65
https://github.com/tigerhawkvok
name: Philip Ka

name: Philip Kahn, bio: CoStar Group; independent work w/ @SSARHERPS , @ArctosDB , @AmphibiaWeb

PGP: https://pgp.mit.edu/pks/lookup?op=vindex&search=0x93E0B76BD90C509
User tigerhawkvok: commits 8 , changed lines + 543 - 61
https://github.com/jveitchmichaelis
name: Josh Veitch-Michaelis, bio: Computer vision engineer. Postdoc building AI-equipped drones for conservation.
User jveitchmichaelis: commits 7 , changed lines + 34 - 10
https://github.com/IlyaOvodov
name: IlyaOvodov, bio: 
User IlyaOvodov: commits 7 , changed lines + 233 - 123
https://github.com/aughey
name: John Aughey, bio: 
User aughey: commits 4 , changed lines + 70 - 230
https://github.com/adujardin
name: Aymeric Dujardin, bio: 
User adujardin: commits 4 , changed lines + 32 - 5
https://github.com/acxz
name: , bio: 
User acxz: commits 4 , changed lines + 67 - 27
https://github.com/judwhite
name: Jud White, bio: 
User judwhite: commits 3 , changed lines + 6 - 4
https://github.com/davidssmith
name: David Smith, bio: Computa


https://github.com/RonLek/FastV2C-HandNet

https://github.com/surajiyer/Recommender-Systems

https://github.com/pietz/unet-keras

https://github.com/tikyau/bidaf-quick-test

https://github.com/Cadene/recipe1m.bootstrap.pytorch

https://github.com/jxz542189/project_category

https://github.com/akilhylton/simpleNALU-tf

https://github.com/CaseyPan/Chinese-Preprocessing

https://github.com/google/dopamine
https://github.com/psc-g
name: Pablo Samuel Castro, bio: señor swesearcher at Google Brain. Musician.
User psc-g: commits 61 , changed lines + 1,343,433 - 130,127
https://github.com/evcu
name: Utku Evci, bio: 2018 Google Brain AI Resident 
M.Sc at NYU Courant. 
Machine Learning Learner.
User evcu: commits 5 , changed lines + 712 - 187
https://github.com/theSparta
name: Rishabh Agarwal, bio: Researcher at Google Brain
User theSparta: commits 4 , changed lines + 21 - 8
https://github.com/tlwillke
name: Ted Willke, bio: 
User tlwillke: commits 2 , changed lines + 2 - 10
https://github.com/

KeyboardInterrupt: 

### Getting list of repositories and authors from the file

In [77]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [79]:
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

1148


In [86]:
author_dict['https://github.com/google/tf_mesh_renderer']

['Kyle Genova',
 'Forrester Cole',
 'Aaron Maschinot',
 'Aaron Sarna',
 'Daniel Vlasic',
 'William T. Freeman']

In [84]:
filename_mini = 'scraped_data_mini.txt' # use full version later

In [89]:
paper_data = {}

with open(filename_mini) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            current_repo = line[0]
            contributors = [line[1:]]                    

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

IndexError: list index out of range

In [90]:
paper_data

{'https://github.com/google/tf_mesh_renderer': {'Kyle Genova': ['Kyle_Genova',
   '2',
   '18',
   '1']},
 'https://github.com/twke18/Adaptive_Affinity_Fields': {'Tsung-Wei Ke': ['Tsung-Wei_Ke',
   '2',
   '24',
   '6'],
  'Jyh-Jing Hwang': ['Jyh-Jing_Hwang', '9', '31', '26']},
 'https://github.com/VladyslavUsenko/basalt-mirror': {'Vladyslav Usenko': ['Vladyslav_Usenko',
   '227',
   '66,951',
   '10,162'],
  'Nikolaus Demmel': ['Nikolaus_Demmel', '24', '662', '1,219']},
 'https://github.com/new-okaerinasai/bwgan_pytorch': {},
 'https://github.com/ManuelVs/NeuralNetworks': {},
 'https://github.com/ruotianluo/DiscCaptioning': {},
 'https://github.com/nitika-verma/FeaStNet': {'Nitika Verma': ['Nitika_Verma',
   '5',
   '344',
   '11']},
 'https://github.com/tfolkman/deep-learning-experiments': {},
 'https://github.com/csliangdu/FSASL': {},
 'https://github.com/loshchil/SGDR': {'Ilya Loshchilov': ['Ilya_Loshchilov',
   '4',
   '461',
   '2']},
 'https://github.com/minhhuu291/Logo-Recognit

### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master